<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Representations of second-quantized states</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Stephane Virally</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

### Imports and definitions

In [19]:
# 
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy.constants as sc
import qutip as qu
import imageio
from matplotlib.image import imsave
from random import uniform
from matplotlib import colors as colors
from numpy.polynomial.hermite import hermval
from scipy.special import gamma, factorial
from numpy.lib.scimath import sqrt, log, power, arccos, arcsin, arctanh
from numpy import exp, sin, cos, tan, sinh, cosh, tanh, arctan, arcsinh, arccosh
sp.init_printing()
fS = 24;
fH = 6.0;
rcctxh = {"font.family": "STIXGeneral", "font.size": fS, "legend.fontsize": fS, "figure.titlesize": fS,
         "axes.titlesize": fS, "axes.labelsize": fS, "xtick.labelsize": fS, "ytick.labelsize": fS,
         "savefig.format": "pdf", "figure.figsize": [fH*1.618, fH]};
rcctxv = {"font.family": "STIXGeneral", "font.size": fS, "legend.fontsize": fS, "figure.titlesize": fS,
         "axes.titlesize": fS, "axes.labelsize": fS, "xtick.labelsize": fS, "ytick.labelsize": fS,
         "savefig.format": "pdf", "figure.figsize": [fH, fH*1.618]};
rcctxs = {"font.family": "STIXGeneral", "font.size": fS, "legend.fontsize": fS, "figure.titlesize": fS,
         "axes.titlesize": fS, "axes.labelsize": fS, "xtick.labelsize": fS, "ytick.labelsize": fS,
         "savefig.format": "pdf", "figure.figsize": [fH, fH]};

### Utility functions

In [2]:
# RGB image from figure
def fig2img(canvas, colortype='RGB', croph=0, cropw=0):
    """
    Extracts RGB image from figure
    canvas: canvas of the figure
    colortype: RGB or alpha-RGB (ARGB)
    croph: crop height (figure croped around center, to height croph)
    cropw: crop width (figure croped around center, to width cropw)
    """
    canvas.draw()
    (width, height) = canvas.get_width_height()
    if colortype == 'ARGB':
        imgarray = np.frombuffer(canvas.tostring_argb(), dtype='uint8').reshape((height, width, 4))
    else:
        imgarray = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape((height, width, 3))
    if (croph>0) and (croph<height):
        crophT = (height-croph)//2
        crophB = height-croph-crophT
        imgarray = imgarray[crophT:-crophB,:,:]
    if (cropw>0) and (cropw<width):
        cropwL = (width-cropw)//2
        cropwR = width-cropw-cropwL
        imgarray = imgarray[:,cropwL:-cropwR,:]   
    return imgarray

# Color normalization around definite midpoint
class mpNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vmid=0., clip=False):
        self.vmid = vmid
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.vmid, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Hermite function series from Hermite polynomial series hermval
def hermfval(x, cs):
    ns = np.arange(len(cs))
    return hermval(x, cs/sqrt((2**ns)*factorial(ns)))*exp(-x**2/2)/(sc.pi)**0.25

# Draw random data from given probability density function
def randomfromfun(data, frequency, numberofpoints):
    cumulativefrequency = np.zeros(len(frequency))
    cumulativefrequency[0] = frequency[0]
    for i in range(1, len(frequency)):
        cumulativefrequency[i] = cumulativefrequency[i-1]+frequency[i]+1e-12
    res = np.zeros(numberofpoints)
    for i in range(numberofpoints):
        y = uniform(0, cumulativefrequency[-1])
        res[i] = np.interp(y, cumulativefrequency, data)
    return res

### Second quantized states visualization functions

In [75]:
# Sticks
def plotSticks(S, nmax, yzlim, titlename='', savename=''):
    ns = S.data.toarray().flatten()
    reals = np.real(ns)
    imags = np.imag(ns)
    if nmax > len(ns):
        nmax = len(ns)
    with plt.rc_context(rcctxv):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_facecolor('white')
        ax.plot((0, nmax-1), (0.0,0.0), (0.0,0.0), color='lightslategrey', lw=2)
        for n in range(nmax-1,-1,-1):
            ax.quiver(n, 0., 0., 0., reals[n], imags[n], color='midnightblue', lw=2, arrow_length_ratio=0.2, length=1.5)
            #ax.plot((n, n), (0.0, reals[n]), (0.0, imags[n]), color='steelblue', lw=2)
            #ax.plot((n, n), (reals[n], reals[n]), (imags[n] , imags[n]), marker='o', color='tomato')
        #ax.view_init(elev=-135, azim=5)
        ax.view_init(elev=-120, azim=10)
        ax.set_xlim(-0.5, nmax-0.5)
        ax.set_ylim(-yzlim, yzlim)
        ax.set_zlim(-yzlim, yzlim)
        ax.set_axis_off()
        #plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        if titlename:
            plt.title(titlename)
        image = fig2img(fig.canvas)
        if savename:
            fig.set_dpi(144)
            #fig.canvas.print_png(savename + '.png', transparent=True, bbox='tight', pad=0)
            fig.canvas.print_png(savename + '.png')
        plt.close(fig)
    return image

# Wigner
def plotWigner(S, X, ticks, ticklabels, titlename='', savename='', cmap='bwr'):
    W = qu.wigner(S, X, X)
    with plt.rc_context(rcctxh):
        fig = plt.figure()
        ax = fig.add_subplot(111, aspect='equal')
        ax.set_facecolor('white')
        ctr = ax.contour(X, X, W, 1000, cmap=cmap, norm=mpNormalize())
        #cb = fig.colorbar(ctr, ax=ax)
        #cb.set_ticks(ticks)
        #cb.set_ticklabels(ticklabels)
        #ax.set_axis_off()
        plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        if titlename:
            plt.title(titlename)
        image = fig2img(fig.canvas)
        if savename:
            fig.set_dpi(144)
            #fig.canvas.print_png(savename + '.png', transparent=True, bbox='tight', pad=0)
            fig.canvas.print_png(savename + '.png')
        plt.close(fig)
    return image

# Harmonic oscillator levels
def plotHarmonic(S, nmax, xlim, ylim, titlename='', savename=''):
    ns = S.data.toarray().flatten()
    creals = np.real(ns)
    cimags = np.imag(ns)
    if nmax > len(ns):
        nmax = len(ns)
    k = nmax/xlim**2
    xs = np.linspace(-xlim, xlim, int(100*xlim)+1)
    reals = np.zeros([nmax+1, len(xs)])
    imags = np.zeros([nmax+1, len(xs)])
    for i in range(nmax+1):
        zerosandoneone = np.zeros(i+1, dtype=int)
        zerosandoneone[-1] = 1
        reals[i] = hermfval(xs, creals[:i+1]*zerosandoneone)
        imags[i] = hermfval(xs, cimags[:i+1]*zerosandoneone)
    ws = hermfval(xs, (creals+1j*cimags))
    with plt.rc_context(rcctxs):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_facecolor('white')
        for i in range(nmax+1):
            ax.plot(xs, reals[i]+i, color='midnightblue', lw=1)
            ax.plot(xs, imags[i]+i, color='darkred', lw=1)
        ax.plot(xs, 2*abs(ws)**2-2, color='darkgreen', lw=1)
        ax.plot(xs, k*xs**2, color='lightslategray', lw=1)
        ax.set_xlim(-xlim, xlim)
        ax.set_ylim(-ylim-2, ylim+nmax)
        #ax.set_axis_off()
        plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        if titlename:
            plt.title(titlename)
        image = fig2img(fig.canvas)
        if savename:
            fig.set_dpi(144)
            #fig.canvas.print_png(savename + '.png', transparent=True, bbox='tight', pad=0)
            fig.canvas.print_png(savename + '.png')
        plt.close(fig)
    return image

# Time-like
def plotTime(evS, ts, xlim, titlename='', savename='', color='midnightblue'):
    rs = np.zeros([len(evS.states), 100])
    xs = np.linspace(-xlim, xlim, int(100*xlim)+1)
    for i in range(len(evS.states)):
        ns = evS.states[i].data.toarray().flatten()
        fs = abs(hermfval(xs, ns))**2
        rs[i] = randomfromfun(xs, fs, 100)
    with plt.rc_context(rcctxs):
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_facecolor('white')
        ax.plot(ts, rs, color=color, marker='.', linestyle='None', markersize=1)
        ax.set_xlim(ts[0], ts[-1])
        ax.set_ylim(-xlim, xlim)
        #ax.set_axis_off()
        plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        #plt.xlabel('$t/T$')
        #plt.ylabel('Electric field [arb. un.]')
        if titlename:
            plt.title(titlename)
        fig.set_dpi(144)
        image = fig2img(fig.canvas, croph=640, cropw=640)
        if savename:
            imsave(savename+'.png', image)
        plt.close(fig)
    return image

### Hilbert space and Hamiltonian for a single second quantized frequency mode

In [4]:
# Fock space dimension
N = 40
# Hamiltonian
H = 2*sc.pi*qu.num(40)

### Coherent state

In [32]:
# Creation
Sa = (qu.coherent(N, np.sqrt(3)*1j)).unit()
# Evolution
n_a = 40
max_a = 1
times_a = np.linspace(0.0, max_a*(n_a-1)/n_a, n_a)
evSa = qu.mesolve(H, Sa, times_a, [], [])

In [74]:
# Wigner
X_a = np.linspace(-7,7,351)
W_a = qu.wigner(evSa.states[0], X_a, X_a)
ticks_a = [0., W_a.max()]
ticklabels_a = ['{:.2f}'.format(0.),'{:.2f}'.format(W_a.max())]
wignercoherentimages = [plotWigner(S, X_a, ticks_a, ticklabels_a, 'Coherent state', '', 'bwr') for S in evSa.states]
imageio.mimsave('./wignercoherent.gif', wignercoherentimages, fps=10);
#wignercoherentimages = [plotWigner(S, X_a, ticks_a, ticklabels_a, 'Coherent state', '', 'PRGn') for S in evSa.states]
#imageio.mimsave('./wignercoherentgreen.gif', wignercoherentimages, fps=15);
#wignercoherentimages = [plotWigner(S, X_a, ticks_a, ticklabels_a, 'Coherent state', '', 'RdBu') for S in evSa.states]
#imageio.mimsave('./wignercoherentblue.gif', wignercoherentimages, fps=20);
for i in range(n_a)[::n_a//8]:
    plotWigner(evSa.states[i], X_a, ticks_a, ticklabels_a, '', './wignercoherent_{}'.format(i), 'bwr')
#for i in range(n_a)[::n_a//8]:
#    plotWigner(evSa.states[i], X_a, ticks_a, ticklabels_a, '', './wignercoherentgreen_{}'.format(i), 'PRGn')
#for i in range(n_a)[::n_a//8]:
#    plotWigner(evSa.states[i], X_a, ticks_a, ticklabels_a, '', './wignercoherentblue_{}'.format(i), 'RdBu')

In [ ]:
# Sticks
#stickcoherentimages = [plotSticks(S, 15, 0.8, 'Coherent state') for S in evSa.states]
#imageio.mimsave('./stickcoherent.gif', stickcoherentimages, fps=5)
for i in range(n_a)[::n_a//8]:
    plotSticks(evSa.states[i], 15, 0.6, '', './stickcoherent_{}'.format(i))

In [ ]:
# Harmonic oscillator
#harmoniccoherentimages = [plotHarmonic(S, 8, 6.0, 1.0, 'Coherent state') for S in evSa.states]
#imageio.mimsave('./harmoniccoherent.gif', harmoniccoherentimages, fps=10)
for i in range(n_a)[::n_a//8]:
    plotHarmonic(evSa.states[i], 8, 6.0, 1.0, '', './harmoniccoherent_{}'.format(i))

In [79]:
# Time-like
#doubletimes_a = np.linspace(0.0, 2.0, 801) # Blue
doubletimes_a = np.linspace(0.0, 1.5, 601) # Green
#doubletimes_a = np.linspace(0.0, 1.0, 401) # Red
doubleevSa = qu.mesolve(H, Sa, doubletimes_a, [], [])
#plotTime(doubleevSa, doubletimes_a, 6, '', './timecoherent', 'midnightblue');
plotTime(doubleevSa, doubletimes_a, 6, '', './timecoherentgreen', 'darkgreen');
#plotTime(doubleevSa, doubletimes_a, 6, '', './timecoherentred', 'red');

### Cat state

In [44]:
# Creation
Sc = (qu.coherent(N, np.sqrt(5))+qu.coherent(N, -np.sqrt(5))).unit()
# Evolution
n_c = 40
max_c = 1
times_c = np.linspace(0.0, max_c*(n_c-1)/n_c, n_c)
evSc = qu.mesolve(H, Sc, times_c, [], [])

In [45]:
# Wigner
X_c = np.linspace(-7,7,351)
W_c = qu.wigner(evSc.states[0], X_c, X_c)
ticks_c = [W_c.min(), 0., W_c.max()]
ticklabels_c = ['{:.2f}'.format(W_c.min()),'{:.2f}'.format(0.),'{:.2f}'.format(W_c.max())]
wignercatimages = [plotWigner(S, X_c, ticks_c, ticklabels_c, 'Cat state') for S in evSc.states]
imageio.mimsave('./wignercat.gif', wignercatimages, fps=10)
for i in range(n_c)[::n_c//8]:
    plotWigner(evSc.states[i], X_c, ticks_c, ticklabels_c, '', './wignercat_{}'.format(i))

In [ ]:
# Sticks
#stickcatimages = [plotSticks(S, 15, 0.8, 'Cat state') for S in evSc.states]
#imageio.mimsave('./stickcat.gif', stickcatimages, fps=5)
for i in range(n_c)[::n_c//8]:
    plotSticks(evSc.states[i], 15, 0.6, '', './stickcat_{}'.format(i))

In [ ]:
# Harmonic oscillator
#harmoniccatimages = [plotHarmonic(S, 8, 6.0, 1.0, 'Cat state') for S in evSc.states]
#imageio.mimsave('./harmoniccat.gif', harmoniccatimages, fps=10)
for i in range(n_c)[::n_c//8]:
    plotHarmonic(evSc.states[i], 8, 6.0, 1.0, '', './harmoniccat_{}'.format(i))

In [86]:
# Time-like
#doubletimes_c = np.linspace(0.0, 2.0, 801)
doubletimes_c = np.linspace(0.0, 1.0, 401)
doubleevSc = qu.mesolve(H, Sc, doubletimes_c, [], [])
#plotTime(doubleevSc, doubletimes_c, 6, '', './timecat');
plotTime(doubleevSc, doubletimes_c, 6, '', './timecatred','red');

### Squeezed vacuum

In [47]:
# Creation
Ssv = (qu.squeeze(N, 0.7)*qu.fock(N, 0)).unit()
# Evolution
n_sv = 40
max_sv = 1
times_sv = np.linspace(0.0, max_sv*(n_sv-1)/n_sv, n_sv)
evSsv = qu.mesolve(H, Ssv, times_sv, [], [])

In [48]:
# Wigner
X_sv = np.linspace(-7,7,351)
W_sv = qu.wigner(evSsv.states[0], X_sv, X_sv)
ticks_sv = [0, W_sv.max()]
ticklabels_sv = ['{:.2f}'.format(0),'{:.2f}'.format(W_sv.max())]
wignersqueezedvacimages = [plotWigner(S, X_sv, ticks_sv, ticklabels_sv, 'Squeezed vacuum') for S in evSsv.states]
imageio.mimsave('./wignersqueezedvac.gif', wignersqueezedvacimages, fps=10)
for i in range(n_sv)[::n_sv//8]:
    plotWigner(evSsv.states[i], X_sv, ticks_sv, ticklabels_sv, '', './wignersqueezedvac_{}'.format(i))

In [ ]:
# Sticks
#sticksqueezedvacimages = [plotSticks(S, 15, 0.8, 'Squeezed vacuum') for S in evSsv.states]
#imageio.mimsave('./sticksqueezedvac.gif', sticksqueezedvacimages, fps=5)
for i in range(n_sv)[::n_sv//8]:
    plotSticks(evSsv.states[i], 15, 0.6, '', './sticksqueezedvac_{}'.format(i))

In [ ]:
# Harmonic oscillator
#harmonicsqueezedvacimages = [plotHarmonic(S, 8, 6.0, 1.0, 'Squeezed vacuum') for S in evSsv.states]
#imageio.mimsave('./harmonicsqueezedvac.gif', harmonicsqueezedvacimages, fps=10)
for i in range(n_sv)[::n_sv//8]:
    plotHarmonic(evSsv.states[i], 8, 6.0, 1.0, '', './harmonicsqueezedvac_{}'.format(i))

In [85]:
# Time-like
#doubletimes_sv = np.linspace(0.0, 2.0, 801)
doubletimes_sv = np.linspace(0.0, 1.0, 401)
doubleevSsv = qu.mesolve(H, Ssv, doubletimes_sv, [], [])
#plotTime(doubleevSsv, doubletimes_sv, 6, '', './timesqueezedvac');
plotTime(doubleevSsv, doubletimes_sv, 6, '', './timesqueezedvacred', 'red');

### Amplitude squeezed state

In [50]:
# Creation
Sas = (qu.displace(N, np.sqrt(3))*qu.squeeze(N, 0.7)*qu.fock(N, 0)).unit()
# Evolution
n_as = 40
max_as = 1
times_as = np.linspace(0.0, max_as*(n_as-1)/n_as, n_as)
evSas = qu.mesolve(H, Sas, times_as, [], [])

In [51]:
# Wigner
X_as = np.linspace(-7,7,351)
W_as = qu.wigner(evSas.states[0], X_as, X_as)
ticks_as = [0, W_as.max()]
ticklabels_as = ['{:.2f}'.format(0),'{:.2f}'.format(W_as.max())]
wignerampsqueezedimages = [plotWigner(S, X_as, ticks_as, ticklabels_as, 'Amplitude squeezed') for S in evSas.states]
imageio.mimsave('./wignerampsqueezed.gif', wignerampsqueezedimages, fps=10)
for i in range(n_as)[::n_as//8]:
    plotWigner(evSas.states[i], X_as, ticks_as, ticklabels_as, '', './wignerampsqueezed_{}'.format(i))

In [ ]:
# Sticks
#stickampsqueezedimages = [plotSticks(S, 15, 0.8, 'Amplitude squeezed') for S in evSas.states]
#imageio.mimsave('./stickampsqueezed.gif', stickampsqueezedimages, fps=5)
for i in range(n_as)[::n_as//8]:
    plotSticks(evSas.states[i], 15, 0.6, '', './sticksampsqueezed_{}'.format(i))

In [ ]:
# Harmonic oscillator
#harmonicampsqueezedimages = [plotHarmonic(S, 8, 6.0, 1.0, 'Amplitude squeezed') for S in evSas.states]
#imageio.mimsave('./harmonicampsqueezed.gif', harmonicampsqueezedimages, fps=10)
for i in range(n_as)[::n_as//8]:
    plotHarmonic(evSas.states[i], 8, 6.0, 1.0, '', './harmonicampsqueezed_{}'.format(i))

In [52]:
# Time-like
doubletimes_as = np.linspace(0.0, 2.0, 801)
doubleevSas = qu.mesolve(H, Sas, doubletimes_as, [], [])
plotTime(doubleevSas, doubletimes_as, 6, '', './timeampsqueezed');

### Phase squeezed state

In [53]:
# Creation
Sps = (qu.displace(N, np.sqrt(3))*qu.squeeze(N, -0.7)*qu.fock(N, 0)).unit()
# Evolution
n_ps = 40
max_ps = 1
times_ps = np.linspace(0.0, max_ps*(n_ps-1)/n_ps, n_ps)
evSps = qu.mesolve(H, Sps, times_ps, [], [])

In [54]:
# Wigner
X_ps = np.linspace(-7,7,351)
W_ps = qu.wigner(evSps.states[0], X_ps, X_ps)
ticks_ps = [0, W_ps.max()]
ticklabels_ps = ['{:.2f}'.format(0),'{:.2f}'.format(W_ps.max())]
wignerphasesqueezedimages = [plotWigner(S, X_ps, ticks_ps, ticklabels_ps, 'Phase squeezed') for S in evSps.states]
imageio.mimsave('./wignerphasesqueezed.gif', wignerphasesqueezedimages, fps=10)
for i in range(n_ps)[::n_ps//8]:
    plotWigner(evSps.states[i], X_ps, ticks_ps, ticklabels_ps, '', './wignerphasesqueezed_{}'.format(i))

In [ ]:
# Sticks
#stickphasesqueezedimages = [plotSticks(S, 15, 0.8, 'Phase squeezed') for S in evSps.states]
#imageio.mimsave('./stickphasesqueezed.gif', stickphasesqueezedimages, fps=5)
for i in range(n_ps)[::n_ps//8]:
    plotSticks(evSps.states[i], 15, 0.6, '', './sticksphasesqueezed_{}'.format(i))

In [ ]:
# Harmonic oscillator
#harmonicphasesqueezedimages = [plotHarmonic(S, 8, 6.0, 1.0, 'Phase squeezed') for S in evSps.states]
#imageio.mimsave('./harmonicphasesqueezed.gif', harmonicphasesqueezedimages, fps=10)
for i in range(n_ps)[::n_ps//8]:
    plotHarmonic(evSps.states[i], 8, 6.0, 1.0, '', './harmonicphasesqueezed_{}'.format(i))

In [55]:
# Time-like
doubletimes_ps = np.linspace(0.0, 2.0, 801)
doubleevSps = qu.mesolve(H, Sps, doubletimes_ps, [], [])
plotTime(doubleevSps, doubletimes_ps, 6, '', './timephasesqueezed');

### Fock state

In [56]:
# Creation (this is an eigenstate of the Hamiltonian)
Sn = (qu.fock(N, 3)).unit()

In [59]:
# Wigner
X_n = np.linspace(-7,7,351)
W_n = qu.wigner(Sn, X_n, X_n)
ticks_n = [W_n.min(), 0, W_n.max()]
ticklabels_n = ['{:.2f}'.format(W_n.min()),'{:.2f}'.format(0),'{:.2f}'.format(W_n.max())]
wignerfockimage = [plotWigner(Sn, X_n, ticks_n, ticklabels_n, 'Fock N=3')]
imageio.mimsave('./wignerfock.gif', wignerfockimage)
plotWigner(Sn, X_n, ticks_n, ticklabels_n, '', './wignerfock');

In [ ]:
# Sticks
plotSticks(Sn, 15, 0.6, '', './stickfock');

In [ ]:
# Harmonic oscillator
plotHarmonic(Sn, 8, 6.0, 1.0, '', './harmonicfock');

In [58]:
# Time-like
doubletimes_n = np.linspace(0.0, 2.0, 801)
doubleevSn = qu.mesolve(H, Sn, doubletimes_n, [], [])
plotTime(doubleevSn, doubletimes_n, 6, '', './timefock');

### Vacuum state

In [60]:
# Creation (this is an eigenstate of the Hamiltonian)
Sv = (qu.fock(N, 0)).unit()

In [62]:
# Wigner
X_v = np.linspace(-7,7,351)
W_v = qu.wigner(Sv, X_v, X_v)
ticks_v = [W_v.min(), 0, W_v.max()]
ticklabels_v = ['{:.2f}'.format(W_v.min()),'{:.2f}'.format(0),'{:.2f}'.format(W_v.max())]
wignervacuumimage = [plotWigner(Sv, X_v, ticks_v, ticklabels_v, 'Vacuum')]
imageio.mimsave('./wignervacuum.gif', wignervacuumimage)
plotWigner(Sv, X_v, ticks_v, ticklabels_v, '', './wignervacuum');

In [66]:
# Sticks
plotSticks(Sv, 15, 1.1, '', './stickvacuum');

In [64]:
# Harmonic oscillator
plotHarmonic(Sv, 8, 6.0, 1.0, '', './harmonicvacuum');

In [80]:
# Time-like
doubletimes_v = np.linspace(0.0, 2.0, 801)
doubleevSv = qu.mesolve(H, Sv, doubletimes_n, [], [])
#plotTime(doubleevSv, doubletimes_v, 6, '', './timevacuum');
plotTime(doubleevSv, doubletimes_v, 6, '', './timevacuumred', 'red');